In [1]:
import torch
import tensorflow as tf
import numpy as np

from transformers import TFT5EncoderModel, TFT5Model
from transformers import T5Config

In [3]:
from tokenization_enc_dec_encn import EncDecTokenizer
tokenizer = EncDecTokenizer('./encn/vocab.txt')

In [5]:
config = T5Config(
    vocab_size=tokenizer.vocab_size,
#     n_positions=self.n_positions,
    d_model=4096,
    d_ff=10240,
    d_kv=4096 // 64,
    num_layers=24,
    num_heads=64,
    relative_attention_num_buckets=32,
    dropout_rate=0.0,
    initializer_factor=1.0,
    eos_token_id=tokenizer.eod_id,
    bos_token_id=tokenizer.pad_id,
    pad_token_id=tokenizer.pad_id,
    decoder_start_token_id=tokenizer.pad_id,
    feed_forward_proj='gated-gelu',
    tie_word_embeddings=False
)

In [6]:
model = TFT5EncoderModel(config)

In [7]:
strs = tf.TensorSpec(shape=[None, None],
                     dtype=tf.int32,
                     name="input_1")
model._set_inputs(strs)

In [8]:
out = model(input_ids=tf.constant([[1] * 512]))

In [9]:
len(model.variables)

219

In [10]:
out.keys()

odict_keys(['last_hidden_state'])

In [11]:
def get_weight(name):
    return state_dict[name].numpy()


def get_block_weight(n, t='encoder', name=False, dim=4096):
    weights = []
    for k, v in state_dict.items():
        if t in k and f'blocks.{n}.' in k:
            # pytorch和tensorflow版本的weights是矩阵转置的
            w = v.numpy()
            if 'self_attn.project' in k:
                w0, w1, w2 = w[:dim, :], w[dim:dim*2, :], w[dim*2:, :]
                w0 = np.transpose(w0)
                w1 = np.transpose(w1)
                w2 = np.transpose(w2)
                weights.append((k, w0))
                weights.append((k, w1))
                weights.append((k, w2))
            elif 'cross_attn.project_q' in k:
                w = np.transpose(w)
                weights.append((k, w))
            elif 'cross_attn.project_kv' in k:
                w0, w1 = w[:dim, :], w[dim:, :]
                w0 = np.transpose(w0)
                w1 = np.transpose(w1)
                weights.append((k, w0))
                weights.append((k, w1))
            else:
                if 'dense' in k:
                    w = np.transpose(w)
                weights.append((k, w))
    if 'relative_attention_bias' in weights[3][0]:
        weights = weights[3:4] + weights[:3] + weights[4:]
    if not name:
        weights = [x[1] for x in weights]
    return weights

In [13]:
state_dict = torch.load('../converted.zip')

In [14]:
model_new_weights = [get_weight('word_embeds.weight')]
for i in range(24):
    model_new_weights += get_block_weight(i, t='encoder')
model_new_weights += [get_weight('encoder.final_layernorm.weight')]

In [15]:
len(model_new_weights)

219

In [16]:
len(model.variables)

219

In [17]:
for k, v in state_dict.items():
    print(k, v.shape)

word_embeds.weight torch.Size([51968, 4096])
lm_head.weight torch.Size([51968, 4096])
encoder.word_embeds.weight torch.Size([51968, 4096])
encoder.final_layernorm.weight torch.Size([4096])
encoder.blocks.0.self_attn.self_attn.project.weight torch.Size([12288, 4096])
encoder.blocks.0.self_attn.self_attn.relative_attention_bias.weight torch.Size([32, 64])
encoder.blocks.0.self_attn.self_attn.dense.weight torch.Size([4096, 4096])
encoder.blocks.0.self_attn.layer_norm.weight torch.Size([4096])
encoder.blocks.0.ff.dense_relu_dense.wi_0.weight torch.Size([10240, 4096])
encoder.blocks.0.ff.dense_relu_dense.wi_1.weight torch.Size([10240, 4096])
encoder.blocks.0.ff.dense_relu_dense.wo.weight torch.Size([4096, 10240])
encoder.blocks.0.ff.layer_norm.weight torch.Size([4096])
encoder.blocks.1.self_attn.self_attn.project.weight torch.Size([12288, 4096])
encoder.blocks.1.self_attn.self_attn.dense.weight torch.Size([4096, 4096])
encoder.blocks.1.self_attn.layer_norm.weight torch.Size([4096])
encoder.

In [18]:
for k in model_new_weights:
    print(k.shape)

(51968, 4096)
(32, 64)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096,)
(4096, 10240)
(4096, 10240)
(10240, 4096)
(4096,)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096,)
(4096, 10240)
(4096, 10240)
(10240, 4096)
(4096,)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096,)
(4096, 10240)
(4096, 10240)
(10240, 4096)
(4096,)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096,)
(4096, 10240)
(4096, 10240)
(10240, 4096)
(4096,)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096,)
(4096, 10240)
(4096, 10240)
(10240, 4096)
(4096,)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096,)
(4096, 10240)
(4096, 10240)
(10240, 4096)
(4096,)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096,)
(4096, 10240)
(4096, 10240)
(10240, 4096)
(4096,)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096,)
(4096, 10240)
(4096, 10240)
(10240, 4096)
(4096,)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096,)
(4096, 10240)
(4096, 10240)
(10240, 4

In [19]:
for x in model.variables:
    print(x.name, x.shape)

shared/shared/weight:0 (51968, 4096)
tf_t5encoder_model/encoder/block_._0/layer_._0/SelfAttention/relative_attention_bias/embeddings:0 (32, 64)
tf_t5encoder_model/encoder/block_._0/layer_._0/SelfAttention/q/kernel:0 (4096, 4096)
tf_t5encoder_model/encoder/block_._0/layer_._0/SelfAttention/k/kernel:0 (4096, 4096)
tf_t5encoder_model/encoder/block_._0/layer_._0/SelfAttention/v/kernel:0 (4096, 4096)
tf_t5encoder_model/encoder/block_._0/layer_._0/SelfAttention/o/kernel:0 (4096, 4096)
tf_t5encoder_model/encoder/block_._0/layer_._0/layer_norm/weight:0 (4096,)
tf_t5encoder_model/encoder/block_._0/layer_._1/DenseReluDense/wi_0/kernel:0 (4096, 10240)
tf_t5encoder_model/encoder/block_._0/layer_._1/DenseReluDense/wi_1/kernel:0 (4096, 10240)
tf_t5encoder_model/encoder/block_._0/layer_._1/DenseReluDense/wo/kernel:0 (10240, 4096)
tf_t5encoder_model/encoder/block_._0/layer_._1/layer_norm/weight:0 (4096,)
tf_t5encoder_model/encoder/block_._1/layer_._0/SelfAttention/q/kernel:0 (4096, 4096)
tf_t5encoder_

In [20]:
assert len(model_new_weights) == len(model.variables)

In [21]:
model.set_weights(model_new_weights)

In [22]:
input_text = '''当地时间9月6日是美国劳工节，但就在这一天，上千万美国劳动者却陷入新的困境。因为美国政府为疫情期间失业者提供的主要救助同日到期，而且白宫表示没有进一步延长救助的计划。
在德尔塔变异株已把美国推入新一轮疫情的背景下，失业救济的突然“断供”意味着有上千万美国人将全部或部分失去他们的生活来源。'''
input_ids = tf.constant([tokenizer.encode(input_text)])

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.563 seconds.
Prefix dict has been built successfully.


In [23]:
%%time
out = model(
    input_ids
)

CPU times: user 1min 18s, sys: 10min 15s, total: 11min 34s
Wall time: 17.1 s


In [24]:
out.keys()

odict_keys(['last_hidden_state'])

In [25]:
out['last_hidden_state'].shape

TensorShape([1, 102, 4096])

In [26]:
model.save('cpm_2_0_tf_encoder')

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


INFO:tensorflow:Assets written to: cpm_2_0_tf_encoder/assets


INFO:tensorflow:Assets written to: cpm_2_0_tf_encoder/assets


In [27]:
!du -sh 'cpm_2_0_tf_encoder'

19G	cpm_2_0_tf_encoder


In [28]:
# !pip install -U tf2onnx

In [29]:
!python -m tf2onnx.convert \
    --large_model \
    --saved-model 'cpm_2_0_tf_encoder' \
    --opset 13 --output \
    cpm_2_0_encoder_onnx.zip

2021-09-21 01:22:26.813910: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-21 01:22:26.813940: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/opt/conda/lib/python3.8/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2021-09-21 01:22:29.840769: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-21 01:22:29.840797: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-21 01:22:29.84

2021-09-21 01:30:22,750 - INFO - folding node using tf type=ConcatV2, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._9/layer_._1/DenseReluDense/wo/Tensordot/concat
2021-09-21 01:30:22,750 - INFO - folding node using tf type=ConcatV2, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._10/layer_._0/SelfAttention/q/Tensordot/concat
2021-09-21 01:30:22,750 - INFO - folding node using tf type=ConcatV2, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._10/layer_._0/SelfAttention/k/Tensordot/concat
2021-09-21 01:30:22,750 - INFO - folding node using tf type=ConcatV2, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._10/layer_._0/SelfAttention/v/Tensordot/concat
2021-09-21 01:30:22,750 - INFO - folding node using tf type=ConcatV2, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._10/layer_._0/SelfAttention/o/Tensordot/concat
2021-09-21 01:30:22,751 - INFO - folding node using tf type=ConcatV2, name=StatefulPartitionedCall/

2021-09-21 01:30:22,908 - INFO - folding node using tf type=Identity, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._0/layer_._0/SelfAttention/k/Tensordot/ReadVariableOp
2021-09-21 01:30:23,052 - INFO - folding node using tf type=Identity, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._0/layer_._0/SelfAttention/v/Tensordot/ReadVariableOp
2021-09-21 01:30:23,197 - INFO - folding node using tf type=Identity, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._0/layer_._0/SelfAttention/o/Tensordot/ReadVariableOp
2021-09-21 01:30:23,341 - INFO - folding node using tf type=Identity, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._0/layer_._1/layer_norm/ReadVariableOp
2021-09-21 01:30:23,342 - INFO - folding node using tf type=Identity, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._0/layer_._1/DenseReluDense/wi_0/Tensordot/ReadVariableOp
2021-09-21 01:30:23,701 - INFO - folding node using tf type=Identity, name=S

2021-09-21 01:30:31,197 - INFO - folding node using tf type=Identity, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._5/layer_._0/SelfAttention/k/Tensordot/ReadVariableOp
2021-09-21 01:30:31,341 - INFO - folding node using tf type=Identity, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._5/layer_._0/SelfAttention/v/Tensordot/ReadVariableOp
2021-09-21 01:30:31,486 - INFO - folding node using tf type=Identity, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._5/layer_._0/SelfAttention/o/Tensordot/ReadVariableOp
2021-09-21 01:30:31,630 - INFO - folding node using tf type=Identity, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._5/layer_._1/layer_norm/ReadVariableOp
2021-09-21 01:30:31,630 - INFO - folding node using tf type=Identity, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._5/layer_._1/DenseReluDense/wi_0/Tensordot/ReadVariableOp
2021-09-21 01:30:31,993 - INFO - folding node using tf type=Identity, name=S

2021-09-21 01:30:39,557 - INFO - folding node using tf type=Identity, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._10/layer_._0/SelfAttention/k/Tensordot/ReadVariableOp
2021-09-21 01:30:39,701 - INFO - folding node using tf type=Identity, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._10/layer_._0/SelfAttention/v/Tensordot/ReadVariableOp
2021-09-21 01:30:39,850 - INFO - folding node using tf type=Identity, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._10/layer_._0/SelfAttention/o/Tensordot/ReadVariableOp
2021-09-21 01:30:39,995 - INFO - folding node using tf type=Identity, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._10/layer_._1/layer_norm/ReadVariableOp
2021-09-21 01:30:39,995 - INFO - folding node using tf type=Identity, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._10/layer_._1/DenseReluDense/wi_0/Tensordot/ReadVariableOp
2021-09-21 01:30:40,359 - INFO - folding node using tf type=Identity, n

2021-09-21 01:30:47,990 - INFO - folding node using tf type=Identity, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._15/layer_._0/SelfAttention/k/Tensordot/ReadVariableOp
2021-09-21 01:30:48,134 - INFO - folding node using tf type=Identity, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._15/layer_._0/SelfAttention/v/Tensordot/ReadVariableOp
2021-09-21 01:30:48,279 - INFO - folding node using tf type=Identity, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._15/layer_._0/SelfAttention/o/Tensordot/ReadVariableOp
2021-09-21 01:30:48,424 - INFO - folding node using tf type=Identity, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._15/layer_._1/layer_norm/ReadVariableOp
2021-09-21 01:30:48,425 - INFO - folding node using tf type=Identity, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._15/layer_._1/DenseReluDense/wi_0/Tensordot/ReadVariableOp
2021-09-21 01:30:48,784 - INFO - folding node using tf type=Identity, n

2021-09-21 01:30:57,055 - INFO - folding node using tf type=Identity, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._20/layer_._0/SelfAttention/k/Tensordot/ReadVariableOp
2021-09-21 01:30:57,200 - INFO - folding node using tf type=Identity, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._20/layer_._0/SelfAttention/v/Tensordot/ReadVariableOp
2021-09-21 01:30:57,349 - INFO - folding node using tf type=Identity, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._20/layer_._0/SelfAttention/o/Tensordot/ReadVariableOp
2021-09-21 01:30:57,497 - INFO - folding node using tf type=Identity, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._20/layer_._1/layer_norm/ReadVariableOp
2021-09-21 01:30:57,497 - INFO - folding node using tf type=Identity, name=StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._20/layer_._1/DenseReluDense/wi_0/Tensordot/ReadVariableOp
2021-09-21 01:30:57,899 - INFO - folding node using tf type=Identity, n

2021-09-21 01:35:44,424 - INFO - replacing einsum node 'StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._17/layer_._0/SelfAttention/einsum/Einsum' by its decomposed version, name of the last node 'Identity__3847'.
2021-09-21 01:35:44,431 - INFO - replacing einsum node 'StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._11/layer_._0/SelfAttention/einsum/Einsum' by its decomposed version, name of the last node 'Identity__3898'.
2021-09-21 01:35:44,439 - INFO - replacing einsum node 'StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._6/layer_._0/SelfAttention/einsum/Einsum' by its decomposed version, name of the last node 'Identity__3949'.
2021-09-21 01:35:44,446 - INFO - replacing einsum node 'StatefulPartitionedCall/tf_t5encoder_model/encoder/block_._2/layer_._0/SelfAttention/einsum/Einsum' by its decomposed version, name of the last node 'Identity__4000'.
2021-09-21 01:35:44,453 - INFO - replacing einsum node 'StatefulPartitionedCall/tf_t5encoder_model/encoder

In [30]:
!ls

bert			       test_tf_encoder.ipynb
configuration_enc_dec.py       test_tokenizer.ipynb
convert_multi_to_single.ipynb  tokenization_enc_dec_encn.py
cpm_2_0_encoder_onnx.zip       tokenization_enc_dec.py
cpm_2_0_tf_encoder	       to_pytorch_encoder_only.ipynb
encn			       to_pytorch.ipynb
model.py		       to_tensorflow_encoder_only.ipynb
onnx-test.ipynb		       to_tensorflow.ipynb
__pycache__		       Untitled.ipynb
README.md		       vocab.txt
test_model_generation.ipynb


In [31]:
!mkdir -p cpm_2_0_encoder_onnx && cd cpm_2_0_encoder_onnx && unzip ../cpm_2_0_encoder_onnx.zip

Archive:  ../cpm_2_0_encoder_onnx.zip
 extracting: __MODEL_PROTO.onnx      
 extracting: StatefulPartitionedCall_tf_t5encoder_model_encoder_final_layer_norm_ReadVariableOp__393_0  
 extracting: StatefulPartitionedCall_tf_t5encoder_model_encoder_block_._9_layer_._1_layer_norm_ReadVariableOp__263_1  
 extracting: StatefulPartitionedCall_tf_t5encoder_model_encoder_block_._9_layer_._1_DenseReluDense_wo_Tensordot_ReadVariableOp__266_2  
 extracting: StatefulPartitionedCall_tf_t5encoder_model_encoder_block_._9_layer_._1_DenseReluDense_wi_1_Tensordot_ReadVariableOp__265_3  
 extracting: StatefulPartitionedCall_tf_t5encoder_model_encoder_block_._9_layer_._1_DenseReluDense_wi_0_Tensordot_ReadVariableOp__264_4  
 extracting: StatefulPartitionedCall_tf_t5encoder_model_encoder_block_._9_layer_._0_layer_norm_ReadVariableOp__258_5  
 extracting: StatefulPartitionedCall_tf_t5encoder_model_encoder_block_._9_layer_._0_SelfAttention_v_Tensordot_ReadVariableOp__261_6  
 extracting: StatefulPartitionedCal

 extracting: StatefulPartitionedCall_tf_t5encoder_model_encoder_block_._3_layer_._0_SelfAttention_q_Tensordot_ReadVariableOp__205_61  
 extracting: StatefulPartitionedCall_tf_t5encoder_model_encoder_block_._3_layer_._0_SelfAttention_o_Tensordot_ReadVariableOp__208_62  
 extracting: StatefulPartitionedCall_tf_t5encoder_model_encoder_block_._3_layer_._0_SelfAttention_k_Tensordot_ReadVariableOp__206_63  
 extracting: StatefulPartitionedCall_tf_t5encoder_model_encoder_block_._23_layer_._1_layer_norm_ReadVariableOp__389_64  
 extracting: StatefulPartitionedCall_tf_t5encoder_model_encoder_block_._23_layer_._1_DenseReluDense_wo_Tensordot_ReadVariableOp__392_65  
 extracting: StatefulPartitionedCall_tf_t5encoder_model_encoder_block_._23_layer_._1_DenseReluDense_wi_1_Tensordot_ReadVariableOp__391_66  
 extracting: StatefulPartitionedCall_tf_t5encoder_model_encoder_block_._23_layer_._1_DenseReluDense_wi_0_Tensordot_ReadVariableOp__390_67  
 extracting: StatefulPartitionedCall_tf_t5encoder_model_

 extracting: StatefulPartitionedCall_tf_t5encoder_model_encoder_block_._18_layer_._0_SelfAttention_v_Tensordot_ReadVariableOp__342_123  
 extracting: StatefulPartitionedCall_tf_t5encoder_model_encoder_block_._18_layer_._0_SelfAttention_q_Tensordot_ReadVariableOp__340_124  
 extracting: StatefulPartitionedCall_tf_t5encoder_model_encoder_block_._18_layer_._0_SelfAttention_o_Tensordot_ReadVariableOp__343_125  
 extracting: StatefulPartitionedCall_tf_t5encoder_model_encoder_block_._18_layer_._0_SelfAttention_k_Tensordot_ReadVariableOp__341_126  
 extracting: StatefulPartitionedCall_tf_t5encoder_model_encoder_block_._17_layer_._1_layer_norm_ReadVariableOp__335_127  
 extracting: StatefulPartitionedCall_tf_t5encoder_model_encoder_block_._17_layer_._1_DenseReluDense_wo_Tensordot_ReadVariableOp__338_128  
 extracting: StatefulPartitionedCall_tf_t5encoder_model_encoder_block_._17_layer_._1_DenseReluDense_wi_1_Tensordot_ReadVariableOp__337_129  
 extracting: StatefulPartitionedCall_tf_t5encoder_

 extracting: StatefulPartitionedCall_tf_t5encoder_model_encoder_block_._11_layer_._1_DenseReluDense_wi_1_Tensordot_ReadVariableOp__283_183  
 extracting: StatefulPartitionedCall_tf_t5encoder_model_encoder_block_._11_layer_._1_DenseReluDense_wi_0_Tensordot_ReadVariableOp__282_184  
 extracting: StatefulPartitionedCall_tf_t5encoder_model_encoder_block_._11_layer_._0_layer_norm_ReadVariableOp__276_185  
 extracting: StatefulPartitionedCall_tf_t5encoder_model_encoder_block_._11_layer_._0_SelfAttention_v_Tensordot_ReadVariableOp__279_186  
 extracting: StatefulPartitionedCall_tf_t5encoder_model_encoder_block_._11_layer_._0_SelfAttention_q_Tensordot_ReadVariableOp__277_187  
 extracting: StatefulPartitionedCall_tf_t5encoder_model_encoder_block_._11_layer_._0_SelfAttention_o_Tensordot_ReadVariableOp__280_188  
 extracting: StatefulPartitionedCall_tf_t5encoder_model_encoder_block_._11_layer_._0_SelfAttention_k_Tensordot_ReadVariableOp__278_189  
 extracting: StatefulPartitionedCall_tf_t5encode